In [ ]:
import sys
sys.path.append('../code')

%run envconfig.py

from imports import *
from run import *

In [ ]:
# dump
dump = "/Volumes/photo"
staging_home = ""
server_home = "/Volumes"
ignore = []
replace = False
mode = 'move'
staging = {}
server = {"HOME": server_home,
          "image": f"{server_home}/photo",
          "video": f"{server_home}/video",
          "audio": f"{server_home}/photo",
          "archive": f"{server_home}/documents"}

arguments = Arguments(dump, staging, server, ignore, replace, mode)   

In [ ]:
# Search for files
files = glob.glob(f"/Volumes/photo/201[7-9]*/**/*.*", recursive=True)

In [ ]:
# describe files (string ops, fast)
shallow = filedesc_shallow(files=files)

# grub stats for each file (slower)
deep = filedesc_deep(files=files)

# Combine
description = pd.merge(shallow, deep, how="outer", on="abspath_src")

In [ ]:
d = description.sort_values(by=["basename_src"])
d["dirname_src"] = d["abspath_src"].apply(os.path.dirname).apply(os.path.basename)

# Look for same name & size
subset=["filename_src", "st_size"]

where = d.duplicated(subset=subset, keep="last")

# Mark deletion
d.loc[where,"delete"] = True

d = d.sort_values(by=subset)
to_delete = list(d[d["delete"] == True]["abspath_src"])

In [ ]:
# Check all duplicates
d[d.duplicated(subset=subset, keep=False)].sort_values(by="filename_src")

In [ ]:
# Remove them
[os.remove(path) for path in to_delete]

## Move CR2 files over to /Volumes/photo_raw

In [ ]:
files = glob.glob(f"/Volumes/photo/**/*.CR2", recursive=True)

In [ ]:
dest = [f.replace("/photo/", "/photo_raw/") for f in files]

In [ ]:
dest

In [ ]:
# Create destination directories if they don't exist
dst_dirs = pd.Series(dest).apply(lambda x: os.path.dirname(x))
dst_dirs_unique = dst_dirs.unique()
_ = [os.makedirs(d, exist_ok=True) for d in dst_dirs_unique]
    

In [ ]:
# Move files
for s,d in zip(files, dest):
    shutil.move(s, d, copy_function=shutil.copy2)